In [1]:

!pip install fastapi uvicorn nest_asyncio python-multipart pandas ollama



   ------------- -------------------------- 1/3 [starlette]
   -------------------------- ------------- 2/3 [fastapi]
   -------------------------- ------------- 2/3 [fastapi]
   ---------------------------------------- 3/3 [fastapi]



In [2]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, File, UploadFile
from io import StringIO
import pandas as pd
import ollama

# Allow FastAPI to run inside Jupyter
nest_asyncio.apply()


In [3]:
client = ollama.Client(host='http://127.0.0.1:11434')  # Change host if needed 

def summarize_csv(df: pd.DataFrame, max_rows=5):
    summary = f"The dataset has {df.shape[0]} rows and {df.shape[1]} columns.\n\n"
    summary += "Columns:\n"
    for col in df.columns:
        summary += f"- {col} (type: {df[col].dtype})\n"
    sample = df.head(max_rows).to_csv(index=False)
    return summary + "\nSample data:\n" + sample

def ask_ollama(question, context, df):
    sales_total = None
    for col in df.columns:
        if 'sale' in col.lower():
            try:
                sales_total = df[col].astype(float).sum()
            except:
                pass
            break

    sales_info = f"\n\nTotal Sales (calculated): {sales_total}" if sales_total is not None else ""
    prompt = f"""You are a data analyst. Here's a dataset summary and a sample:\n\n{context}{sales_info}\n\nNow answer this question about the dataset: {question}"""

    response = client.chat(model="llama3", messages=[
        {"role": "user", "content": prompt}
    ])
    return response["message"]["content"]


In [4]:
app = FastAPI()

@app.post("/analyze_csv/")
async def analyze_csv(file: UploadFile = File(...), question: str = "What are the key insights?"):
    try:
        content = await file.read()
        df = pd.read_csv(StringIO(content.decode("utf-8")))

        summary = summarize_csv(df)
        result = ask_ollama(question, summary, df)

        return {
            "status": "success",
            "question": question,
            "summary": summary,
            "response": result
        }

    except Exception as e:
        return {"status": "error", "message": str(e)}


In [ ]:
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [6696]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:57063 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:57063 - "GET /openapi.json HTTP/1.1" 200 OK
